The distress signal leads you to a giant waterfall! Actually, hang on - the signal seems like it's coming from the waterfall itself, and that doesn't make any sense. However, you do notice a little path that leads behind the waterfall.

Correction: the distress signal leads you behind a giant waterfall! There seems to be a large cave system here, and the signal definitely leads further inside.

As you begin to make your way deeper underground, you feel the ground rumble for a moment. Sand begins pouring into the cave! If you don't quickly figure out where the sand is going, you could quickly become trapped!

Fortunately, your familiarity with analyzing the path of falling material will come in handy here. You scan a two-dimensional vertical slice of the cave above you (your puzzle input) and discover that it is mostly air with structures made of rock.

Your scan traces the path of each solid rock structure and reports the x,y coordinates that form the shape of the path, where x represents distance to the right and y represents distance down. Each path appears as a single line of text in your scan. After the first point of each path, each point indicates the end of a straight horizontal or vertical line to be drawn from the previous point. For example:

```
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
```

This scan means that there are two paths of rock; the first path consists of two straight lines, and the second path consists of three straight lines. (Specifically, the first path consists of a line of rock from 498,4 through 498,6 and another line of rock from 498,6 through 496,6.)

The sand is pouring into the cave from point 500,0.

Drawing rock as `#`, air as `.`, and the source of the sand as `+`, this becomes:

```
  4     5  5
  9     0  0
  4     0  3
0 ......+...
1 ..........
2 ..........
3 ..........
4 ....#...##
5 ....#...#.
6 ..###...#.
7 ........#.
8 ........#.
9 #########.
```

Sand is produced one unit at a time, and the next unit of sand is not produced until the previous unit of sand comes to rest. A unit of sand is large enough to fill one tile of air in your scan.

A unit of sand always falls down one step if possible. If the tile immediately below is blocked (by rock or sand), the unit of sand attempts to instead move diagonally one step down and to the left. If that tile is blocked, the unit of sand attempts to instead move diagonally one step down and to the right. Sand keeps moving as long as it is able to do so, at each step trying to move down, then down-left, then down-right. If all three possible destinations are blocked, the unit of sand comes to rest and no longer moves, at which point the next unit of sand is created back at the source.

So, drawing sand that has come to rest as o, the first unit of sand simply falls straight down and then stops:

```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......o.#.
#########.
```

The second unit of sand then falls straight down, lands on the first one, and then comes to rest to its left:

```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
.....oo.#.
#########.
```

After a total of five units of sand have come to rest, they form this pattern:

```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
......o.#.
....oooo#.
#########.
```

After a total of 22 units of sand:

```
......+...
..........
......o...
.....ooo..
....#ooo##
....#ooo#.
..###ooo#.
....oooo#.
...ooooo#.
#########.
```

Finally, only two more units of sand can possibly come to rest:

```
......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.
```

Once all 24 units of sand shown above have come to rest, all further sand flows out the bottom, falling into the endless void. Just for fun, the path any new sand takes before falling forever is shown here with ~:

```
.......+...
.......~...
......~o...
.....~ooo..
....~#ooo##
...~o#ooo#.
..~###ooo#.
..~..oooo#.
.~o.ooooo#.
~#########.
~..........
~..........
~..........
```

***Using your scan, simulate the falling sand. How many units of sand come to rest before sand starts flowing into the abyss below?***

In [9]:
import numpy as np
input_file  = open("data/2022-12-14-input.txt")
input_lines = input_file.readlines()
rock_paths = [[list(map(int, x.split(','))) for x in line.strip().split(' -> ')]
              for line in input_lines]
rock_x = [loc[0] for locs in rock_paths for loc in locs]
rock_y = [loc[1] for locs in rock_paths for loc in locs]
print(f'x goes from {min(rock_x)} to {max(rock_x)}')
print(f'y goes from {min(rock_y)} to {max(rock_y)}')

x goes from 457 to 541
y goes from 13 to 168


In [64]:
abyss_start = max(rock_y) + 1
x_adj = min(rock_x) - 1
cross_section = np.full((max(rock_x) - x_adj + 2, abyss_start + 1), '.')

def PrintCrossSection(cross_section):
    np.apply_along_axis(lambda pixel_row: print("".join(pixel_row)), 0, cross_section)

In [65]:
sand_start = [500 - x_adj, 0]
cross_section[sand_start[0], sand_start[1]] = '+'
for rock_path in rock_paths:
    cur_rock_x = rock_path[0][0] - x_adj
    cur_rock_y = rock_path[0][1]
    cross_section[cur_rock_x, cur_rock_y] = '#'
    for [next_rock_x, next_rock_y] in rock_path[1:]:
        next_rock_x -= x_adj
        while cur_rock_x != next_rock_x or cur_rock_y != next_rock_y:
            cur_rock_x += np.sign(next_rock_x - cur_rock_x)
            cur_rock_y += np.sign(next_rock_y - cur_rock_y)
            cross_section[cur_rock_x, cur_rock_y] = '#'

PrintCrossSection(cross_section)

............................................+..........................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
................................

In [66]:
def SandRestingPosition(sand_start, cross_section, abyss_start):
    [sand_x, sand_y] = sand_start
    match sand_start:
        case [_, sand_y] if sand_y == abyss_start:
            return (sand_x, sand_y)
        case [sand_x, sand_y] if cross_section[sand_x, sand_y + 1] == '.':
            return SandRestingPosition([sand_x, sand_y + 1], cross_section, abyss_start)
        case [sand_x, sand_y] if cross_section[sand_x - 1, sand_y + 1] == '.':
            return SandRestingPosition([sand_x - 1, sand_y + 1], cross_section, abyss_start)
        case [sand_x, sand_y] if cross_section[sand_x + 1, sand_y + 1] == '.':
            return SandRestingPosition([sand_x + 1, sand_y + 1], cross_section, abyss_start)
    return (sand_x, sand_y)

n_sand = 0
while True:
    rest_x, rest_y = SandRestingPosition(sand_start, cross_section, abyss_start)
    if rest_y == abyss_start:
        break
    cross_section[rest_x, rest_y] = 'o'
    n_sand += 1

PrintCrossSection(cross_section)
print(f'{n_sand} units of sand come to rest before sand starts falling into the abyss')

............................................+..........................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
.......................................................................................
............................................o..........................................
...........................................ooo.........................................
..........................................ooooo........................................
................................

You realize you misread the scan. There isn't an endless void at the bottom of the scan - there's floor, and you're standing on it!

You don't have time to scan the floor, so assume the floor is an infinite horizontal line with a y coordinate equal to two plus the highest y coordinate of any point in your scan.

In the example above, the highest y coordinate of any point is 9, and so the floor is at y=11. (This is as if your scan contained one extra rock path like -infinity,11 -> infinity,11.) With the added floor, the example above now looks like this:

```
        ...........+........
        ....................
        ....................
        ....................
        .........#...##.....
        .........#...#......
        .......###...#......
        .............#......
        .............#......
        .....#########......
        ....................
<-- etc #################### etc -->
```

To find somewhere safe to stand, you'll need to simulate falling sand until a unit of sand comes to rest at 500,0, blocking the source entirely and stopping the flow of sand into the cave. In the example above, the situation finally looks like this after 93 units of sand come to rest:

```
............o............
...........ooo...........
..........ooooo..........
.........ooooooo.........
........oo#ooo##o........
.......ooo#ooo#ooo.......
......oo###ooo#oooo......
.....oooo.oooo#ooooo.....
....oooooooooo#oooooo....
...ooo#########ooooooo...
..ooooo.......ooooooooo..
#########################
```

***Using your scan, simulate the falling sand until the source of the sand becomes blocked. How many units of sand come to rest?***

In [75]:
floor = max(rock_y) + 2
min_x = min(rock_x) - floor - 2
max_x = max(rock_x) + floor + 2
cross_section = np.full((max_x - min_x, floor + 1), '.')

sand_start = [500 - min_x, 0]
cross_section[sand_start[0], sand_start[1]] = '+'
for rock_path in rock_paths:
    cur_rock_x = rock_path[0][0] - min_x
    cur_rock_y = rock_path[0][1]
    cross_section[cur_rock_x, cur_rock_y] = '#'
    for [next_rock_x, next_rock_y] in rock_path[1:]:
        next_rock_x -= min_x
        while cur_rock_x != next_rock_x or cur_rock_y != next_rock_y:
            cur_rock_x += np.sign(next_rock_x - cur_rock_x)
            cur_rock_y += np.sign(next_rock_y - cur_rock_y)
            cross_section[cur_rock_x, cur_rock_y] = '#'

for i in range(np.shape(cross_section)[0]):
    cross_section[i, floor] = '#'

n_sand = 0
while True:
    n_sand += 1
    rest_x, rest_y = SandRestingPosition(sand_start, cross_section, floor + 5)
    if [rest_x, rest_y] == sand_start:
        break
    cross_section[rest_x, rest_y] = 'o'

print(f'{n_sand} units of sand come to rest before the source is blocked')

27426 units of sand come to rest before the source is blocked
